# CRIM Intervals:  Cadences

### What Can You Do With this Notebook?

* Find **cadences**, reporting type, tone, cadential voice functions (roles) and other information about the piece
* Count and analyze distribution of cadences

#### View Score Excerpts with Verovio in the NB


* It is also possible to display the results of the Cadence
    classifier in the Notebook with Verovio. Each excerpt is
    two measures long:  the measure of the final tone of the cadence
    and the previous measure.

* The function also displays metadata about each excerpt, drawn from the
    cadence results dataframe:  piece ID, composer, title, measures, type of
    cadence, beat of the bar in which the final tone is heard, and evaded
    status.
    
* View the cadence table:
   
> `piece.cadences()`
    
* To show the cadences with Verovio, then just:

>`piece.verovioCadences()`

* Note that pink warning messages in the output can be ignored!
   
* If you prefer to create the cadence table, then filter it in some way before passing to Verovio:

>`cadences = piece.cadences()`

(After filtering):

> `piece.verovioCadences(cadences)`




In [136]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
import glob as glob
import os
from IPython.display import SVG
from pathlib import Path
import verovio
MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)

else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


In [137]:
def verovio_print_piece(piece):
    # check path and import MEI or XML

    if piece.path.startswith('Music_Files/'):
        text_file = open(piece.path, "r")
        fetched_mei_string = text_file.read()
    else:
        response = requests.get(piece.path)
        fetched_mei_string = response.text
    # start the verovio toolkit and load the file there
    tk = verovio.toolkit()
    tk.loadData(fetched_mei_string)
    tk.setScale(30)
    tk.setOption( "pageHeight", "1500" )
    tk.setOption( "pageWidth", "3000" )


    tk.redoLayout()
    # get the number of pages and display the music
    count = tk.getPageCount()
    for c in range(1, count + 1):
        music = tk.renderToSVG(c)
        print("File Name: ", piece.file_name)
        print(piece.metadata['composer'])
        print(piece.metadata['title'])
        display(HTML(music))

## B. Importing a Piece

In [138]:
def verovio_MorleyCadences(piece):
        """
    

        """
        if piece.path.startswith('Music_Files/'):
            text_file = open(piece.path, "r")
            fetched_mei_string = text_file.read()
        else:
            response = requests.get(piece.path)
            fetched_mei_string = response.text
        tk = verovio.toolkit()
        tk.loadData(fetched_mei_string)
        tk.setScale(30)
        tk.setOption( "pageHeight", "1500" )
        tk.setOption( "pageWidth", "3000" )
        morleyCadences = piece.morleyCadences()
        morleyCadences = piece.detailIndex(morleyCadences).reset_index()
        for cad in morleyCadences.index:
            c_meas = morleyCadences.loc[cad]["Measure"]
            c_beat = morleyCadences.loc[cad]["Beat"]
            low = c_meas-1
            high = c_meas
            mr = str(low) + "-" + str(high)
            mdict = {'measureRange': mr}

            # select verovio measures and redo layout
            tk.select(str(mdict))
            tk.redoLayout()

            # get the number of pages and display the music
            print("Results:")
            count = tk.getPageCount()
            for c in range(1, count + 1):
                music = tk.renderToSVG(c)
                print("File Name: ", piece.file_name)
                print(piece.metadata['composer'])
                print(piece.metadata['title'])
                print("Cadence End Measure:", c_meas)
                print("Beat: ", c_beat)
                display(HTML(music))

### B.1 Import a Piece and Check Title

In [11]:
# Select a prefix:
# prefix = 'https://crimproject.org/mei/'
prefix = 'Music_Files/'
# just add the CRIM Piece ID here
mei_file = 'Morley_1597_074_03.musicxml'
# combine strings and import
url = prefix + mei_file
piece = importScore(url)
print(piece.metadata)

Previously imported piece detected.
{'title': 'Morley_1597_074_03', 'composer': 'Morley'}


In [145]:
# GET EVERYTHING AND PRINT VEROVIO
# for name in glob.glob('Music_Files/*'):
#     piece = importScore(name)
#     verovio_print_piece(piece)


### Cadential Voice Functions--What they Mean

In [143]:
# print(ImportedPiece.cvfs.__doc__)

## Select Number of Voices and Import

In [144]:
number_voices = 3
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        corpus_list.append(name)
        print(name  + 'has been added to your list')
#     else:
#         print('sorry, ' + name  + 'is not a three-voice piece')

corpus = CorpusBase(corpus_list)       
        
        


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Music_Files/Morley_1597_127_2.musicxmlhas been added to your list
Previously imported piece detected.
Music_Files/Morley_1597_127_3.musicxmlhas been added to your list
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Music_Files/Morley_1597_127_8.musicxmlhas been added to your list
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Music_Files/Morley_1597_128_01.musicxmlhas been added to your list
Previously imported piece detected.
Previously impo

### Classify the Cadences as Corpus

Here is what the code should look like:

> func = ImportedPiece.cadences  
list_of_dfs = corpus.batch(func=func, kwargs={'keep_keys': True}, metadata=True)  
cadence_results = pd.concat(list_of_dfs, ignore_index=False)  
cadence_results['Validation'] = ""  
cadence_results['Comments'] = ""  
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'Pattern', 'Key', 'CadType', 'Tone','CVFs',
                'LeadingTones', 'Sounding', 'Low','RelLow','RelTone',
                'Progress','SinceLast','ToNext', 'Validation', 'Comments']  
cadence_results = combined_df[col_list]  
cadence_results   

### The usual Cadence Tool

In [146]:

func = ImportedPiece.cadences
list_of_dfs = corpus.batch(func=func)
# func2 = ImportedPiece.detailIndex
# list_dfs_det = corpus.batch(func=func2, kwargs={'df': list_of_dfs, 'offset' : True}, metadata=True)
cadence_results = pd.concat(list_of_dfs, ignore_index=False)

cadence_results = cadence_results.sort_values(['Title', 'Progress'])
cadence_results = cadence_results.drop(['Low', 'RelLow', 'TSig', 'SinceLast', 'ToNext'], axis=1)
cadence_results.reset_index(inplace=True)
cadence_results.rename(columns = {'index':'Offset'}, inplace=True)
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'Offset', 'Progress', 'CadType', 'Tone','CVFs', 'LeadingTones', 'Sounding']
cadence_results = cadence_results[col_list]
cadence_results.head()

,Composer,Title,Measure,Beat,Offset,Progress,CadType,Tone,CVFs,LeadingTones,Sounding
0,Morley,Morley_1597_068,7,3.0,52.0,0.388060,Authentic,G,TCB,1.0,3.0
1,Morley,Morley_1597_068,10,3.0,76.0,0.567164,Authentic,G,tCB,1.0,3.0
2,Morley,Morley_1597_068,13,1.0,96.0,0.716418,Authentic,A,TCB,1.0,3.0
3,Morley,Morley_1597_068,14,1.0,104.0,0.776119,Clausula Vera,G,CT,1.0,3.0
4,Morley,Morley_1597_068,15,1.0,112.0,0.835821,Clausula Vera,D,CT,0.0,3.0


### Summarize by Tone, Type, and Functions

* You can easily summarize the data in various ways:

    >`cadences.groupby(['Tone', 'CadType', 'CVFs']).size().reset_index(name='counts')`

In [141]:
cadence_summary = cadence_results.groupby(['Title', 'Tone', 'CadType', 'CVFs']).size().reset_index(name='counts')
cadence_summary.to_csv('Morley_a3_Cad_Summary_Table.csv')
cadence_summary

,Title,Tone,CadType,CVFs,counts
0,Morley_1597_068,A,Authentic,TCB,1
1,Morley_1597_068,D,Clausula Vera,CT,1
2,Morley_1597_068,G,Authentic,TCB,2
3,Morley_1597_068,G,Authentic,tCB,1
4,Morley_1597_068,G,Clausula Vera,CT,2
5,Morley_1597_127_3,C,Authentic,CtB,1
6,Morley_1597_127_3,C,Clausula Vera,CT,1
7,Morley_1597_127_3,G,Authentic,tCB,1
8,Morley_1597_127_4,G,Evaded Clausula Vera,tC,1
9,Morley_1597_127_7,G,Evaded Clausula Vera,tC,1


### C.4.  Display Results with Verovio

#### For the Corpus
>number_voices = 3
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        piece.verovioCadences()
        
#### For Just One Piece

>prefix = 'Music_Files/'
music_file = 'Morley_1597_074_03.musicxml'
url = prefix + music_file
piece = importScore(url)
print(piece.metadata)

In [147]:
# FOR ONE PIECE
prefix = 'Music_Files/'
music_file = 'Morley_1597_074_03.musicxml'
url = prefix + music_file
piece = importScore(url)
# piece.verovioCadences()

Previously imported piece detected.


In [ ]:
## Print the Whole Corpus according to Voice Filter
## Note you must repeat the import here
number_voices = 5
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        piece.verovioCadences()

### The Morley Cadences as Table

In [135]:
func = ImportedPiece.morleyCadences
list_of_dfs = corpus.batch(func=func, metadata=True)
func2 = ImportedPiece.detailIndex
list_dfs_det = corpus.batch(func=func2, kwargs={'progress': True, 'offset': True,'df': list_of_dfs}, metadata=True)
morleyCadence_results = pd.concat(list_dfs_det, ignore_index=False)
morleyCadence_results = morleyCadence_results.fillna(' ')
morleyCadence_results = morleyCadence_results.reset_index()
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'Offset', 'Progress', '1', '2', '3']
morleyCadence_results = morleyCadence_results[col_list]

morleyCadence_results= morleyCadence_results.sort_values(["Title", 'Progress'])
morleyCadence_results


,Composer,Title,Measure,Beat,Offset,Progress,1,2,3
17,Morley,Morley_1597_068,3,1.000000,16.0,0.119403,Morley Cadence,Morley Cadence,
18,Morley,Morley_1597_068,4,1.000000,24.0,0.179104,Morley Cadence,,
19,Morley,Morley_1597_068,7,1.000000,48.0,0.358209,Morley Cadence,,
20,Morley,Morley_1597_068,7,2.000000,50.0,0.373134,,Morley Cadence,
21,Morley,Morley_1597_068,8,1.000000,56.0,0.417910,,Morley Cadence,
22,Morley,Morley_1597_068,9,3.000000,68.0,0.507463,Morley Cadence,,
23,Morley,Morley_1597_068,10,3.000000,76.0,0.567164,Morley Cadence,Morley Cadence,
24,Morley,Morley_1597_068,12,4.000000,94.0,0.701493,Morley Cadence,,
25,Morley,Morley_1597_068,13,1.000000,96.0,0.716418,,Morley Cadence,
26,Morley,Morley_1597_068,14,1.000000,104.0,0.776119,,Morley Cadence,


### Print Morley Cadences

In [142]:
## Print the Whole Corpus according to Voice Filter
## Note you must repeat the import here
number_voices = 3
corpus_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    notes = piece.notes()
    if len(notes.columns) == number_voices:
        verovio_MorleyCadences(piece)

Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_127_3
Morley
Morley_1597_127_3
Cadence End Measure: 4
Beat:  1.0


Results:
File Name:  Morley_1597_127_3
Morley
Morley_1597_127_3
Cadence End Measure: 6
Beat:  1.0


Results:
File Name:  Morley_1597_127_3
Morley
Morley_1597_127_3
Cadence End Measure: 8
Beat:  1.0


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_128_01
Morley
Morley_1597_128_01
Cadence End Measure: 3
Beat:  1.0


Results:
File Name:  Morley_1597_128_01
Morley
Morley_1597_128_01
Cadence End Measure: 3
Beat:  2.0


...

Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_128_05
Morley
Morley_1597_128_05
Cadence End Measure: 2
Beat:  1.0


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_129_2
Morley
Morley_1597_129_2
Cadence End Measure: 1
Beat:  4.0


[Warning] Measure range start for selection '0-1' could not be found.


Previously imported piece detected.
Results:
File Name:  Morley_1597_129_3
Morley
Morley_1597_129_3
Cadence End Measure: 2
Beat:  3.0


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
